In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

from tkinter import *
import tkinter as tk
import win32gui
from PIL import ImageGrab, Image, ImageTk

In [10]:
model = load_model('mnist-4-5.h5')   #5 is the best till now

def predict_digit(img):
    #resize image to 75x75 pixels
    img = img.resize((100, 100))
    
    #convert rgb to grayscale
    img = img.convert('L')
    img = np.array(img)
    
#     print('Image to be predicted...')
#     plt.imshow(img, cmap = 'gray')
#     plt.show()
    
    #reshaping to support our model input and normalizing
    img = img.reshape(1, 100, 100, 1).astype('float32')
    img = img/255.0
    

    #predicting the class
    res = model.predict([img])
#     print(res)
    digits_res = res[0].flatten()
    digits = digits_res.argsort()[-2:][::-1]
#     print(digits)
    digits_acc = [digits_res[digits[0]], digits_res[digits[1]]]
#     print(digits_acc)
    
    boxes = res[1].flatten()
    div = 100
    box = np.array([boxes[1], (boxes[0]*100 - 28)/div, (boxes[1]*100 + 28)/div, boxes[0],
                boxes[3], (boxes[2]*100 - 28)/div, (boxes[3]*100 + 28)/div, boxes[2]])
    boxes = box.reshape((2, 4))
#     print(boxes)

    return digits, digits_acc, boxes


class App(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)

        self.x = self.y = 0
        
        # Creating elements
        self.title('Handwritten Digit Recognition and Localization')
        self.canvas1 = tk.Canvas(self, width=500, height=500, bg = 'black', cursor="cross")
        self.label1 = tk.Label(self, text="Draw..", font=("Helvetica", 48))
        self.label2 = tk.Label(self, text="Draw..", font=("Helvetica", 48))
        self.classify_btn = tk.Button(self, text = "Recognise", command = self.classify_handwriting)   
        self.button_clear = tk.Button(self, text = "Clear", command = self.clear_all)
       
    # Grid structure
        self.canvas1.grid(row=0, column=0, pady=0, sticky=W, rowspan = 2, columnspan = 1)
        self.label1.grid(row=0, column=1,pady=0, padx=0)
        self.label2.grid(row = 1, column = 1, pady = 0, padx = 0)
        self.classify_btn.grid(row=2, column=1, pady=2, padx = 2)
        self.button_clear.grid(row=2, column=0, pady=2)
        
        self.canvas1.bind("<B1-Motion>", self.draw_lines) # event handler for mouse events

    def clear_all(self):
        self.canvas1.delete("all")
        
    def classify_handwriting(self):
        # code to convert drawing on canvas to an image
        HWND = self.canvas1.winfo_id()        # get the handle of the canvas
        rect = win32gui.GetWindowRect(HWND)  # get the coordinate of the canvas
        self.im = ImageGrab.grab(rect)
    
        # predict what the image is...
        digits, acc, boxes = predict_digit(self.im)
        
        w, h = self.im.size
        
        box01 = boxes[0][0] * h   # height (y_min)
        box02 = boxes[0][1] * w   # width (x_min)
        box03 = boxes[0][2] * h   # height (y_max)
        box04 = boxes[0][3] * w   # width (x_max)
        
        box11 = boxes[1][0] * h   # height (y_min)
        box12 = boxes[1][1] * w   # width (x_min)
        box13 = boxes[1][2] * h   # height (y_max)
        box14 = boxes[1][3] * w   # width (x_max)

        self.img = ImageTk.PhotoImage(self.im)
        self.canvas1.create_image((w/2, h/2), image=self.img, anchor = "center")
        self.canvas1.create_rectangle(box02, box01, box04, box03, outline = 'red')
        self.canvas1.create_rectangle(box12, box11, box14, box13, outline = 'red')
        string1 = str(digits[0])+ ',' + str(int(acc[0] * 100)) + '%'
        string2 = str(digits[1])+ ',' + str(int(acc[1] * 100)) + '%'
        
        self.label1.configure(text = string1)
        if acc[1] > 0.1:
            self.label2.configure(text = string2)
        else:
            self.label2.configure(text = '!')  

#         if box01<=20:
#             self.canvas1.create_text(box02+35, box03 +15, fill = 'red', text = string1, 
#                                  font="Times 20 italic bold")
#         elif box02<=20:
#             self.canvas1.create_text(box04-15, box01-15, fill = 'red', text = string1, 
#                                  font="Times 20 italic bold")
#         else:    
#             self.canvas1.create_text(box02+35, box01-15, fill = 'red', text = string1, 
#                                  font="Times 20 italic bold")
            
#         if box11<=20:
#             self.canvas1.create_text(box12+35, box13 +15, fill = 'red', text = string2, 
#                                  font="Times 20 italic bold")
#         elif box12<=20:
#             self.canvas1.create_text(box14-15, box11-15, fill = 'red', text = string2, 
#                                  font="Times 20 italic bold")
#         else:    
#             self.canvas1.create_text(box12+35, box11-15, fill = 'red', text = string2, 
#                                  font="Times 20 italic bold")

    def draw_lines(self, event):
        # draw on the canvas
        self.x = event.x
        self.y = event.y
        r= 6                              # control the width of strokes
        self.canvas1.create_oval(self.x+r, self.y+r, self.x - r, self.y - r, fill='white',outline = 'white')
       
app = App()
mainloop()
